In [ ]:
import os
from datetime import datetime

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from src.utils import (
    NetCDFToZarrConverter
)
raw_data_path = "../data/raw/seviri"

In [ ]:
# Scaling only performed for time dimension 
# 20MB per chunk is aimed

converter = NetCDFToZarrConverter()

chunk_analysis = converter.analyze_netcdf_files("../data/raw/seviri/hrv_lr2*.nc")

chunk_analysis["recommended_chunks"]

2026-01-29 17:11:29,427 - INFO - Analyzing NetCDF files matching: ../data/raw/seviri/hrv_lr2*.nc
2026-01-29 17:11:29,431 - INFO - Found 56 files
/home/plato/dl_cloudhole/dl_cloudhole/src/utils.py:74: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  dims = dict(ds.dims)
2026-01-29 17:11:31,083 - INFO - Calculated chunk size: 19.97 MB | chunks={'time': 261, 'y': 92, 'x': 109}
2026-01-29 17:11:31,084 - INFO - Analysis complete: {'num_files': 56, 'files': ['hrv_lr200401.nc', 'hrv_lr200411.nc', 'hrv_lr200412.nc', 'hrv_lr200501.nc', 'hrv_lr200511.nc', 'hrv_lr200512.nc', 'hrv_lr200601.nc', 'hrv_lr200611.nc', 'hrv_lr200612.nc', 'hrv_lr200701.nc', 'hrv_lr200711.nc', 'hrv_lr200712.nc', 'hrv_lr200801.nc', 'hrv_lr200811.nc', 'hrv_lr200812.nc', 'hrv_lr200901.nc', 'hrv_lr200911.nc', 'hrv_lr200912.nc', 'h

{'time': 261, 'y': 92, 'x': 109}

In [ ]:
from pathlib import Path
input_file_pattern="../data/raw/seviri/hrv_lr2*.nc"

files = sorted(Path().glob(input_file_pattern))

In [ ]:
"""
- Issue:Segmentation fault (core dumped)
- Reason: parallel=True, chunks='auto' on xr.open_mfdataset
- Solution: set parallel false and chunks to None
"""

converter.convert_multiple_files_to_single_zarr(
    file_pattern = input_file_pattern,
    output_path = f"../data/processed/seviri/hrv_lr{files[0].as_posix()[-9:-5]}_{files[-1].as_posix()[-9:-5]}.zarr",
    custom_chunks = chunk_analysis["recommended_chunks"]
)

2026-01-29 17:07:18,625 - INFO - Converting multiple files to single Zarr: ../data/raw/seviri/hrv_lr2*.nc
2026-01-29 17:07:18,628 - INFO - Found 56 files to convert
2026-01-29 17:07:19,775 - INFO - Writing consolidated Zarr with chunks: {'time': 261, 'y': 92, 'x': 109}


3.955028712749481 GB dataset
Frozen({'time': (261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,

2026-01-29 17:07:49,690 - INFO - Successfully created consolidated Zarr at ../data/processed/seviri/hrv_lr2004_2019.zarr
2026-01-29 17:07:49,692 - INFO - 
2026-01-29 17:07:49,692 - INFO - Zarr Store Information: ../data/processed/seviri/hrv_lr2004_2019.zarr
2026-01-29 17:07:49,693 - INFO - ============================================================
2026-01-29 17:07:49,694 - INFO - /
 ├── hrv (52928, 92, 109) float64
 ├── lat (92, 109) float64
 ├── lon (92, 109) float64
 └── time (52928,) int64
2026-01-29 17:07:49,698 - INFO - ============================================================

